참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 07.003<br>ex07.003

부정정보: 이중적분<br>Statically Indeterminate Beam : Double integration

p. 296

### 문제에서 주어진 변수<br>Given Parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m, a_m = sy.symbols('L_{AB}[m], a[m]', real=True, nonnegative=True)

s_d = {
    L_AB_m: 4,
    a_m: 2,
}


#### 하중<br>Load

In [ ]:
P_N_down = sy.symbols('P[N]', real=True)

s_d.update(
    {
        P_N_down: 5000,
    }
)


#### 잉여구속<br>Residual constraint

In [ ]:
delta_B_m_down = sy.symbols('\delta_{B}[m]', real=True)
s_d[delta_B_m_down] = 30e-3

#### 재료와 단면 특성<br>Material & section properties

In [ ]:
E_Pa, I_m4 = sy.symbols('E[Pa], I[m^{4}]', positive=True)

s_d.update(
    {
        E_Pa: 10 * 10 ** 9,
        I_m4: 20 * 10 ** 6 * sy.Rational(1, 1000) ** 4,
    }
)


#### 자유물체도<br>Free body diagram

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB_m.subs(s_d), 'text':'B'},
]

reaction_list = [
    {'x_m': 0},
    {'x_m': L_AB_m.subs(s_d)},
]

dist_load_list = []

v_load_list = [
    {'x_m':a_m.subs(s_d), 'sign': -1},
]

moment_list = [
    {'x_m': 0, 'direction': 'ccw', 'text': 'M', 'open':'right'},
]

draw_diagrams.draw_beam(L_AB_m.subs(s_d), points_list, reaction_list, v_load_list=v_load_list, dist_load_list=dist_load_list, moment_list=moment_list)

In [ ]:
x_m = sy.symbols('x[m]', nonnegative=True)
x_A_m = 0
x_B_m = L_AB_m


### 아직 알지 못하는 반력<br>Reaction forces unknown yet

In [ ]:
R_A_N_up, M_A_Nm_ccw, R_B_N_up = sy.symbols('R_{A}[N], M_{A}[Nm], R_{B}[N]', real=True)

### 중첩법<br>Superposition

#### 외팔보 $a$ 지점에 가해지는 하중 $P$에 의한 끝점의 처짐<br>Deflection of a cantilever's end point by load $P$ at $a$

p. 279 Table 6.2

$$
\delta_{BP} = \frac{Pa^2}{6EI}(3L-a)
$$

In [ ]:
delta_BP_m_down = (P_N_down * a_m ** 2 / (6 * E_Pa * I_m4)) * (3 * L_AB_m - a_m)

In [ ]:
delta_BP_m_down

변수값을 대입하면<br>Substituting parameters gives

In [ ]:
delta_BP_m_down.subs(s_d)

#### 외팔보 $L$ 지점에 가해지는 반력 $R$에 의한 끝점의 처짐<br>Deflection of a cantilever's end point by reaction force $R$ at $L$

p. 279 Table 6.2

$$
\delta_{BR} = \frac{RL^3}{3EI}
$$

In [ ]:
delta_BR_m_up = (R_B_N_up * L_AB_m ** 3 / (3 * E_Pa * I_m4))

In [ ]:
delta_BR_m_up

변수값을 대입하면<br>Substituting parameters gives

In [ ]:
delta_BR_m_up.subs(s_d)

#### 두 처짐을 중첩함<br>Superpose two $\delta$'s

In [ ]:
super_eq = sy.Eq(delta_BP_m_down - delta_BR_m_up, delta_B_m_down)

In [ ]:
super_eq

#### $R$에 관하여 풂<br>Solve for $R$

In [ ]:
R_B_N_up_sol = sy.solve(super_eq, R_B_N_up)[0]

In [ ]:
R_B_N_up_sol

In [ ]:
R_B_N_up_sol.subs(s_d)

### 평형방정식<br>Equilibrium equation

수직방향<br>Vertical direction

$$
R_A + R_B = P
$$

In [ ]:
fy_eq = sy.Eq(R_A_N_up + R_B_N_up, P_N_down)

In [ ]:
fy_eq

In [ ]:
fy_eq.subs(R_B_N_up, R_B_N_up_sol)

In [ ]:
R_A_N_up_sol = sy.solve(fy_eq.subs(R_B_N_up, R_B_N_up_sol), R_A_N_up)[0]

In [ ]:
R_A_N_up_sol

In [ ]:
R_A_N_up_sol.subs(s_d)

$B$점 중심 회전방향<br>Rotation direction around $B$

$$
M_A - R_A \cdot L + P_N \cdot (L - a)=0
$$

In [ ]:
m_eq = sy.Eq(M_A_Nm_ccw - R_A_N_up * L_AB_m + P_N_down * (L_AB_m - a_m))

In [ ]:
m_eq

In [ ]:
sy.simplify(m_eq.subs(R_A_N_up, R_A_N_up_sol))

In [ ]:
M_A_Nm_ccw_sol = sy.solve(m_eq.subs(R_A_N_up, R_A_N_up_sol), M_A_Nm_ccw)[0]

In [ ]:
sy.simplify(M_A_Nm_ccw_sol)

In [ ]:
M_A_Nm_ccw_sol.subs(s_d)